In [2]:
import os
import getpass
from typing import Dict, List, Any, TypedDict, Optional
from datetime import datetime
from pydantic import BaseModel
from dotenv import load_dotenv
from tavily import TavilyClient
from langchain_groq import ChatGroq
from langchain_core.messages import HumanMessage, SystemMessage
from langgraph.graph import StateGraph

load_dotenv()

if "GROQ_API_KEY" not in os.environ:
    os.environ["GROQ_API_KEY"] = getpass.getpass("Enter your Groq API key: ")

tavily = TavilyClient(api_key=os.getenv("TAVILY_API_KEY"))
llm =  ChatGroq(
    model_name="openai/gpt-oss-120b",
    temperature=0.7
)

In [ ]:
class Article(BaseModel):
    title: str
    url: str
    content: str

class Summary(TypedDict):
    title: str
    summary: str
    url: str

# This defines what information we can store and pass between nodes later
class GraphState(TypedDict):
    articles: Optional[List[Article]] 
    summaries: Optional[List[Summary]] 
    report: Optional[str] 

In [ ]:
class NewsSearcher:
    def search(self) -> List[Article]:
        response = tavily.search(
            query="artificial intelligence and machine learning news", 
            topic="news",
            time_period="1w",
            search_depth="advanced",
            max_results=5
        )
        
        articles = []
        for result in response['results']:
            articles.append(Article(
                title=result['title'],
                url=result['url'],
                content=result['content']
            ))
        
        return articles

In [ ]:
class Summarizer:
    def __init__(self):
        self.system_prompt = """
        You are an AI expert who makes complex topics accessible 
        to general audiences. Summarize this article in 2-3 sentences, focusing on the key points 
        and explaining any technical terms simply.
        """
    
    def summarize(self, article: Article) -> str:
        response = llm.invoke([
            SystemMessage(content=self.system_prompt),
            HumanMessage(content=f"Title: {article.title}\n\nContent: {article.content}")
        ])
        return response.content

In [ ]:
broad_prompt = """ You are a friendly, expert science writer who turns technical AI/ML news into engaging, easy-to-understand stories for the general public. Produce a single weekly AI/ML news roundup in Markdown that a busy reader can scan in under 5 minutes. Follow these rules exactly:

1. Tone & style

- Warm, curious, and upbeat; clear and concise.
- Avoid jargon. If a technical term is necessary, define it in one short phrase.
- Use active voice and short paragraphs (1–3 sentences each).
- Keep the whole report between ~400–800 words.

2. Structure (required)

- Title line with date and a 1-line thematic hook.
- A 2–3 sentence introduction that summarizes the week’s theme and why it matters.
- Main items: list 3–5 news stories. For each item include:
  - Headline (bold)
  - 2–3 sentence plain-language summary of what happened
  - One short "Why it matters" sentence connecting the news to real-world impact
  - Source link labeled “Read more”
  - 3 bullet takeaways (one-line sentences each).
- One closing “What to watch next” sentence with 1 suggested follow-up reading link.

3. Formatting & extras

- Output must be valid Markdown with headings, bold headlines for items, bullet lists for TL;DR and links as inline Markdown links.
- Include an estimated reading time (e.g., “~3 min read”) under the title.
- Where the source is unclear, explicitly state “source reports” and include the URL; do not invent facts.

4. Safety & accuracy

- Do not hallucinate. If a claim is uncertain, mark it as “reported” and include the original link.
- Use only information provided in the summaries you receive; do not add new factual claims.

"""

In [ ]:
class Publisher:
    def create_report(self, summaries: List[Dict]) -> str:
        # prompt = """
        # Create a weekly AI/ML news report for the general public. 
        # Format it with:
        # 1. A brief introduction
        # 2. The main news items with their summaries
        # 3. Links for further reading
        
        # Make it engaging and accessible to non-technical readers.
        # """
        
        prompt = broad_prompt
        
        # Format summaries for the LLM
        summaries_text = "\n\n".join([
            f"Title: {item['title']}\nSummary: {item['summary']}\nSource: {item['url']}"
            for item in summaries
        ])
        
        # Generate report
        response = llm.invoke([
            SystemMessage(content=prompt),
            HumanMessage(content=summaries_text)
        ])
        
        # Add metadata and save
        current_date = datetime.now().strftime("%Y-%m-%d")
        markdown_content = f"""
        Generated on: {current_date}

        {response.content}
        """
        
        filename = f"ai_news_report_{current_date}.md"
        with open(filename, 'w') as f:
            f.write(markdown_content)
        
        return response.content

In [ ]:
def search_node(state: Dict[str, Any]) -> Dict[str, Any]:
    searcher = NewsSearcher()
    state['articles'] = searcher.search() 
    return state

def summarize_node(state: Dict[str, Any]) -> Dict[str, Any]:
    summarizer = Summarizer()
    state['summaries'] = []
    
    for article in state['articles']: # Uses articles from previous node
        summary = summarizer.summarize(article)
        state['summaries'].append({
            'title': article.title,
            'summary': summary,
            'url': article.url
        })
    return state

def publish_node(state: Dict[str, Any]) -> Dict[str, Any]:
    publisher = Publisher()
    report_content = publisher.create_report(state['summaries'])
    state['report'] = report_content
    return state

In [ ]:
def create_workflow() -> StateGraph:
    # Create a workflow (graph) initialized with our state schema
    workflow = StateGraph(state_schema=GraphState)
    
    # Add processing nodes that we will flow between
    workflow.add_node("search", search_node)
    workflow.add_node("summarize", summarize_node)
    workflow.add_node("publish", publish_node)
    
    # Define the flow with edges
    workflow.add_edge("search", "summarize") # search results flow to summarizer
    workflow.add_edge("summarize", "publish") # summaries flow to publisher
    
    # Set where to start
    workflow.set_entry_point("search")
    
    return workflow.compile()

In [ ]:
if __name__ == "__main__":
    # Initialize and run workflow
    workflow = create_workflow()
    final_state = workflow.invoke({
        "articles": None,
        "summaries": None,
        "report": None
    })
    
    # Display results
    print("\n=== AI/ML Weekly News Report ===\n")
    print(final_state['report'])


=== AI/ML Weekly News Report ===

# AI/ML Weekly Roundup – Sep 5 2025  
*AI is moving from labs to boardrooms, reshaping security, industry, and jobs.*  
**~3 min read**

**Intro** – This week’s stories show AI slipping out of research labs and into everyday business decisions. From fire‑wall upgrades to factory floors, companies are betting on machine‑learning (ML) to spot threats, cut waste, and speed up product development. The trend highlights both the promise of smarter operations and the lingering questions about how AI will reshape work.

---

### **AI adoption surges in cybersecurity as threats rise**  
Security teams are racing to add AI and ML tools to their defenses; 61 % say they’ll adopt these technologies within the next year, outpacing most manufacturers.  Companies now view AI‑driven security as essential because cyber attacks rank just behind inflation as the top external risk.  The same tools are also being planned for supply‑chain oversight and product‑quality monit